In [ ]:
library(boot)
library(stats)

## Confidence Intervals

You take a random sample of size n and calculate the sample mean. We use this sample mean as an *estimator* for the true population mean, also known as a "point estimate". A point estimate by itself is of limited usefulness because it does not reveal the uncertainty associated with the estimate or sampling method. Confidence intervals supply us with more information about the estimate by giving us an *interval*, with an associated *level of confidence*, that likely would contain the true population mean.  


* A confidence interval (CI) is an interval of values instead of a single point estimate.
* The level of confidence corresponds to the expected proportion of intervals that will contain the parameter if many confidence intervals are constructed of the same sample size from the same population. Another way of saying this is: 
    * 90% of confidence intervals computed at the 90% confidence level contain the true parameter
    * 95% of confidence intervals computed at the 95% confidence level contain the parameter
    * etc...
* Our uncertainty is about whether our particular CI is one of those that truly contains the true value of the parameter.
* Constructing confidence intervals requires knowledge about the sample distribution - sample mean and either population SD or sample SD. 
* Calculating CIs, theoretically, does not require that the population follow a normal distribution, however the math for such situations can get tricky. For this reason, if we know the population doesn't follow a normal, or we're just unsure, we can instead calculate an asymptotical CI on the *sample distribution for the sample mean* (according to the Central Limit Theorem, the sampling distribution for the sample mean approaches normal).  


General form:

sample statistic $\pm$ margin of error

margin of error $=$ multiplier(standard error)

From Lab 8, we learned standard error $= \frac{\hat\sigma}{\sqrt n}$. The multiplier is calculated using t-statistics from a t-distribution. (There's a lot of theory behind this, but in short - this distribution applies when a test statistic we're interested in would follow a normal distribution *if* the value of a scaling term in the test statistic were known. When it's unknown, the scaling term is replaced by an estimate based on the data (the SEM).) Other statistical methods can be used to calculate the multiplier, but the t-distribution is widely used for when we don't know the true population standar deviation (which we rarely do). 

### Example 
When the population follows a normal

In [52]:
# draw a random sample from the normal distribution
n <- 50
x <- rnorm(n, 0, 1) 
x

[1]  1.19994295  1.62789123 -0.05372978 -0.19973760  2.91284459  0.06786712
 [7]  2.25082651 -0.60634862 -0.42636367  0.56392338 -0.03069558  1.40294121
[13] -0.71438335 -0.71991417 -0.26315693  0.26491253 -0.51913261  1.78317366
[19] -0.34761184  1.55670114  0.27855160  0.54078310  1.42627957  1.11807464
[25]  0.33897263 -0.28365110 -1.04713251 -0.21990898  2.32570336  0.72411910
[31] -1.56208797 -1.31598044 -0.04988923  1.12772718  1.50359510 -1.05621600
[37]  0.20475319  0.20532608  0.26590322 -1.85672749 -0.31633218 -1.73342293
[43]  0.75767547  0.35208737 -0.13341942 -1.89286577  0.19484084 -0.82036901
[49]  0.05988552 -1.28057476

In [ ]:
# histogram of the sample
hist(x)

In [ ]:
# sample mean
mean <- mean(x)
mean

**Question - Is there evidence that our true population mean is different from our sample mean?**

In [ ]:
# standard error of the mean
sem <- sd(x)/sqrt(n)
sem

In [ ]:
# confidence interval with a confidence level of 95%
ci <- c(mean-qt(0.975, n-1)*sem, mean+qt(0.975, n-1)*sem)
ci

How do we interpret this? 

We are 95% confident that the true mean is within the above CI. Because the interval is centered around our sample mean, the answer to the above question is **no, there is no evidence that our true population mean is different from our sample mean**. 

---

**Hypothesis Testing**

In [53]:
# one-sample t-test
t.test(x)


	One Sample t-test

data:  x
t = 0.97495, df = 49, p-value = 0.3344
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.1614248  0.4656508
sample estimates:
mean of x 
 0.152113 


We learned about one-sample t-tests in lecture, which determined whether our population mean was different from a particular value. We can also conduct two-sample t-tests to determine if there is a significant difference between the population means of two groups. Unlike the one-sample t-test, where the t-statistic is calculated using the difference between the sample mean and the value, divided by the standard error ($t = \frac{difference}{standard error}$), the t-statistic for a two-sample t-test is $t = \frac{difference of group averages}{standard error of difference}$. We then compare this t-statistic to a t-value, which depends on our chosen significance value and degrees of freedom. 

In [ ]:
# two-sample t-test
# draw a random sample from the normal distribution
n <- 50
y <- rnorm(n, 0, 1) 
print(mean(y))
t.test(x, y)


---
As mentioned, if the population doesn't follow a normal distribution, or we're unsure, we can construct a CI on the sampling distribution of the sample mean. However, collecting multiple samples, from which we can calculate sample means, is not always feasible. This is where bootstrapping comes in handy.

## Bootstrapping

How it works:

Bootstrapping resamples the original dataset with replacement many thousands of times to create simulated datasets. This process involves drawing random samples from the original dataset. Breaking it down:

* The bootstrap method has an equal probability of randomly drawing each original data point for inclusion in the resampled datasets.
* The procedure can select a data point more than once for a resampled dataset. This property is the “with replacement” aspect of the process.
* The procedure creates resampled datasets that are the same size as the original dataset.

The process ends with your simulated datasets having many different combinations of the values that exist in the original dataset. Each simulated dataset has its own set of sample statistics, such as the mean. Bootstrapping procedures use the distribution of the sample statistics across the simulated samples as the sampling distribution.

Keep in mind that bootstrapping does not create new data. Instead, it treats the original sample as a proxy for the real population and then draws random samples from it. Consequently, the central assumption for bootstrapping is that the original sample accurately represents the actual population. The resampling process creates many possible samples that a study could have drawn. The various combinations of values in the simulated samples collectively provide an estimate of the variability between random samples drawn from the same population. The range of these potential samples allows the procedure to construct confidence intervals and perform hypothesis testing. Importantly, as the sample size increases, bootstrapping converges on the correct sampling distribution under most conditions.

### Example 
When the population doesn't follow a normal (or we're uncertain).

#### Manual bootstrapping:

In [ ]:
# sample data for duck plumage - each row represents a male duck who is a second generation cross of 
# mallard and pintail ducks. Plumage indices were calculates for each duck - ranged from 0 (identical in appearance
# to a mallard) to 20 (identical to a pintail). 

data <- data.frame(weight = c(catsM$Bwt))
n <- length(data$weight)
data

In [ ]:
# histogram of duck plumage indices
hist(data$weight, breaks=10)

In [ ]:
# mean of duck plumage indices
mean <- mean(data$weight)
mean

In [ ]:
# create 30 bootstrap samples and calculate their sample means
bootReps <- 30
means <- c()
for(i in 1:bootReps){
    sample <- sample(data$weight, size = n, replace = T)
    means <- append(means, mean(sample))
}
hist(means)

In [ ]:
# construct the CI for the above sampling distribution of sample means
meanOfMeans <- mean(means)
semOfMeans <- sd(means)/sqrt(bootReps)
ciOfMeans <- c(meanOfMeans-qt(0.975,bootReps-1)*semOfMeans, meanOfMeans+qt(0.975,bootReps-1)*semOfMeans)
ciOfMeans

In [ ]:
# create 100 bootstrap samples and calculate their sample means
bootReps <- 100
means <- c()
for(i in 1:bootReps){
    sample <- sample(data$weight, size = n, replace = T)
    means <- append(means, mean(sample))
}
hist(means)

In [ ]:
# construct the CI for the above sampling distribution of sample means
meanOfMeans <- mean(means)
semOfMeans <- sd(means)/sqrt(bootReps)
ciOfMeans <- c(meanOfMeans-qt(0.975,bootReps-1)*semOfMeans, meanOfMeans+qt(0.975,bootReps-1)*semOfMeans)
ciOfMeans

In [ ]:
# create 1000 bootstrap samples and calculate their sample means
bootReps <- 1000
means <- c()
for(i in 1:bootReps){
    sample <- sample(data$weight, size = n, replace = T)
    means <- append(means, mean(sample))
}
hist(means)

In [ ]:
# construct the CI for the above sampling distribution of sample means
meanOfMeans <- mean(means)
semOfMeans <- sd(means)/sqrt(bootReps)
ciOfMeans <- c(meanOfMeans-qt(0.975,bootReps-1)*semOfMeans, meanOfMeans+qt(0.975,bootReps-1)*semOfMeans)
ciOfMeans

Notice how the sampling distribution in the histogram approximates a normal distribution even though the underlying data distribution is not. This approximation occurs thanks to the central limit theorem. As the sample size increases, the sampling distribution converges on a normal distribution regardless of the underlying data distribution (with a few exceptions).

We can compare our original sample mean to the context of our simulated sample distribution.

#### Using `boot()`:

`boot()` calls the statistic function R times. Each time, it generates a set of random indices, with replacement, from the integers `1:nrow(data)`. These indices are used within the statistic function to select a sample. The statistics are calculated on the sample and the results are accumulated in the bootobject. 

**Note:** The function should include an indices parameter that the `boot()` function can use to select cases for each replication. Refer to example.

In [ ]:
?boot

In [ ]:
# create the `statistic` function for the mean
mean_fun <- function(data,index){
  return(mean(data[index,]))
}

In [ ]:
# run a bootstrap analysis for 30 replicates
res <- boot(data = data, R=30, statistic = mean_fun)
res

In [ ]:
res$t

In [ ]:
plot(res)

In [ ]:
boot.ci(res, conf = 0.95, type = "norm")

In [ ]:
# run a bootstrap analysis for 100 replicates
res <- boot(data = data, R=100, statistic = mean_fun)
res

In [ ]:
plot(res)

In [ ]:
boot.ci(res, conf = 0.95, type = "norm")

In [ ]:
# run a bootstrap analysis for 1000 replicates
res <- boot(data = data, R=1000, statistic = mean_fun)
res

In [ ]:
plot(res)

In [ ]:
boot.ci(res, conf = 0.95, type = "norm")

In [ ]:
mean <- mean(res$t)
sem <- sd(res$t)/sqrt(1000)
ci <- c(mean - qt(0.975, 999)*sem, mean + qt(0.975, 999)*sem)
ci

**Question - Is there a statistically significant difference in weights between male and female cats?**

In [54]:
# is there a statistically significant difference in weights between male and female cats?
library(MASS)
t.test(Bwt ~ Sex, data = cats)


	Welch Two Sample t-test

data:  Bwt by Sex
t = -8.7095, df = 136.84, p-value = 8.831e-15
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.6631268 -0.4177242
sample estimates:
mean in group F mean in group M 
       2.359574        2.900000 
